<a href="https://colab.research.google.com/github/lta36/RL/blob/main/RC2PyTorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import time


# Using GPU Resources

In [ ]:
# change the runtime type to 'gpu' to return true
if (torch.cuda.is_available()):
  print('CUDA AVAILABLE')
  print('Number of devices {}'.format(torch.cuda.device_count()))
  print('First Device name {}'.format(torch.cuda.get_device_name(0)))

else:
  print('NO GPU DETECTED')


CUDA AVAILABLE
Number of devices 1
First Device name Tesla K80


In [ ]:
# initialize a 'device' for ourselves
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# multiple ways to utilize the devices
a = torch.normal(0, 1, size=(1000,1000))
b = torch.normal(0, 1, size=(1000,1000))

a_cuda = a.to(device)
b_cuda = b.cuda()

# a, b no both in GPU memory
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

c_cuda = torch.normal(0, 1, a_cuda.shape).cuda()
# start.record()
start = time.time()
for i in range(300):
  c_cuda = a_cuda + c_cuda@b_cuda
# end.record()
torch.cuda.synchronize() # cuda operations are asynchronous, we must ensure completion

print('GPU TIME {}'.format(time.time() - start))

# bring them back out of gpu memory (for non tensor operations)
d = a.cpu()
e = b.cpu()

c = torch.normal(0, 1, a.shape)
start = time.time()
for i in range(300):
  c = d + c @ e

print('CPU TIME {}'.format(time.time() - start))


GPU TIME 0.4623076915740967
CPU TIME 8.317511558532715


In [ ]:
# Be careful that all tnesors are on the same device or else...
f = torch.normal(0, 1, (10, 10)).cuda()
g = torch.normal(0, 1, (10, 10))
f + g

RuntimeError: ignored

In [ ]:

X = torch.normal(0, 1, size=(10,10))
X

tensor([[ 2.2125,  0.1101,  0.1592,  0.1464, -0.7851, -0.5717, -1.0404,  0.3948,
         -0.8741, -0.1540],
        [-0.5060, -0.2522,  0.8530, -0.7845, -0.5615,  0.4727,  0.7359, -0.6029,
         -0.1937, -1.7146],
        [ 0.3140,  0.6259,  1.0818,  0.8351, -1.9414,  0.7980, -0.6748, -0.4471,
         -1.7709, -0.5060],
        [-3.2152,  1.9721, -0.1604, -0.9541, -0.3643,  0.6880,  1.0778,  0.8099,
          1.6711,  0.5627],
        [ 0.9598, -0.3241, -1.6854, -0.1205, -0.9592, -0.3132,  0.7367, -0.1958,
          1.1983,  1.2918],
        [ 0.3128,  2.3083,  2.5882,  0.4949, -0.3814,  0.6669, -0.4542, -0.1096,
         -1.5927,  0.1255],
        [-0.6593,  1.0749, -0.2221, -0.5929,  0.1506, -0.1506, -0.9415, -0.0319,
          0.2815, -1.4177],
        [ 1.3241, -0.7738,  0.7319,  0.7543, -0.8877,  1.1602, -1.6581,  0.0907,
          1.2870, -0.0309],
        [-1.5790,  1.4794,  1.1337, -2.2275,  0.1356, -0.8547,  0.4506, -0.1759,
         -2.1433, -0.1995],
        [ 0.4458,  

In [ ]:
# Repeat from TF tutorial
Y = np.eye(2)[np.random.choice(2,10)]
Y = torch.tensor(Y, dtype=torch.float32)
Y

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])

In [ ]:
model1 = torch.nn.Sequential(
    torch.nn.Linear(10, 5),
    torch.nn.Tanh(),
    torch.nn.Linear(5, 5),
    torch.nn.ReLU(),
    torch.nn.Linear(5, 2),
    torch.nn.Softmax()
)


In [ ]:
# initialize loss criterion and optimzier as separate objects
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters())

In [ ]:
# run training epoch by epoch
for epoch in range(10):
  epoch_loss = 0.0
  correct = 0
  for i, data in enumerate(zip(X, Y)):
    input, label = data

    #reset the stored parameter gradients
    optimizer.zero_grad()

    #run model
    output = model1(input)
    loss = criterion(output, label)

    #optimization step
    loss.backward()
    optimizer.step()

    epoch_loss += loss
    classified_out = torch.round(output)

    if (torch.equal(classified_out, label)):
      correct += 1

  acc = correct/Y.shape[0]
  print('EPOCH [{}] loss [{}] acc [{}]'.format(epoch, loss, acc))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


EPOCH [0] loss [0.12016613781452179] acc [0.7]
EPOCH [1] loss [0.11737425625324249] acc [0.7]
EPOCH [2] loss [0.11467073857784271] acc [0.7]
EPOCH [3] loss [0.11220872402191162] acc [0.7]
EPOCH [4] loss [0.1098659336566925] acc [0.7]
EPOCH [5] loss [0.10750657320022583] acc [0.7]
EPOCH [6] loss [0.10507810860872269] acc [0.7]
EPOCH [7] loss [0.10255584120750427] acc [0.7]
EPOCH [8] loss [0.09992557764053345] acc [0.7]
EPOCH [9] loss [0.09717822074890137] acc [0.7]



# Customize your own NN!



In [ ]:
class NeuralNet(torch.nn.Module):
  def __init__(self, output_size):
    super().__init__()
    self.linear1 = torch.nn.Linear(10, 5)
    self.activation1 = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(5, 5)
    self.activation2 = torch.nn.ReLU()

    self.output_layer = torch.nn.Linear(5, output_size)
    self.output_activation = torch.nn.Softmax()

    #initialize random weights
    # torch.nn.init.normal_(self.linear1.weight, mean=0.0, std=1.0)
    # torch.nn.init.normal_(self.linear2.weight, mean=0.0, std=1.0)
    # torch.nn.init.normal_(self.output_layer.weight, mean=0.0, std=1.0)\


  def forward(self, inputs):
    x = self.activation1(self.linear1(inputs))
    x = self.activation2(self.linear2(x))

    x = self.output_activation(self.output_layer(x))
    return x



In [ ]:
model2 = NeuralNet(2)
print(list(model2.named_parameters()))

[('linear1.weight', Parameter containing:
tensor([[-0.1250,  0.1025,  0.0731,  0.2993, -0.1616,  0.2586,  0.0217, -0.2611,
         -0.0079, -0.0581],
        [-0.1873,  0.2927,  0.1852,  0.0815,  0.0603,  0.1632, -0.1424,  0.0649,
          0.1465,  0.1581],
        [ 0.2487,  0.1358,  0.2726, -0.2619, -0.3089, -0.1049,  0.1554, -0.2690,
          0.2001, -0.0211],
        [ 0.0308, -0.2634, -0.0473,  0.2982,  0.1203,  0.0688,  0.0506, -0.0553,
          0.2722,  0.0537],
        [-0.0855, -0.0745, -0.2687,  0.0787,  0.0972, -0.2620,  0.0157, -0.0204,
         -0.2964,  0.0421]], requires_grad=True)), ('linear1.bias', Parameter containing:
tensor([ 0.2659, -0.2629, -0.0035, -0.2377,  0.2223], requires_grad=True)), ('linear2.weight', Parameter containing:
tensor([[ 0.0641,  0.0524,  0.0725,  0.0139, -0.0973],
        [-0.1984, -0.3504, -0.0109,  0.1071,  0.4279],
        [ 0.1059, -0.3904, -0.3289,  0.1990, -0.4020],
        [-0.2206,  0.0885, -0.2971, -0.2692,  0.3375],
        [ 0.26

In [ ]:
model2(X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.5727, 0.4273],
        [0.6046, 0.3954],
        [0.5810, 0.4190],
        [0.5618, 0.4382],
        [0.5380, 0.4620],
        [0.6006, 0.3994],
        [0.5993, 0.4007],
        [0.5983, 0.4017],
        [0.6291, 0.3709],
        [0.5436, 0.4564]], grad_fn=<SoftmaxBackward>)